## Intial attempt to download information about the gpt plugins

In [1]:
# from tqdm.auto import tqdm
# import os
# import requests


# # List of URLs for the GPT plugins you are interested in.
# # Replace these with the actual URLs of the plugins.
# plugin_urls = ['wolframalpha.com', 'www.instacart.com', 'www.kayak.com', 'fiscalnote.com', 'www.klarna.com', 'www.joinmilo.com', 'zapier.com', 'www.opentable.com']

# # Directory where the downloaded files will be stored
# download_dir = './downloaded_files'

# # Make sure the download directory exists
# os.makedirs(download_dir, exist_ok=True)

# for url in tqdm(plugin_urls):
#     if not url.startswith("https://"):
#         url = "https://"+url
#     # Construct the URL for the ai-plugin.json file
#     # file_url = os.path.join(url, '.well-known', 'ai-plugin.json')
#     file_url = '/'.join([url, '.well-known', 'ai-plugin.json'])

#     # Make a GET request to the file URL
#     response = requests.get(file_url)

#     # If the request was successful
#     if response.status_code == 200:
#         # Construct the path where the file will be saved
#         file_path = os.path.join(download_dir, f'{url.replace("https://", "")}_ai-plugin.json')

#         # Write the content of the response to a file
#         with open(file_path, 'wb') as file:
#             file.write(response.content)


## Including downloading the information found at the api.url in the ai-plugin.json

In [15]:
# from tqdm.auto import tqdm
# import os
# import json
# import requests
# import shutil
# import re

# websites = [
#     "www.wolframalpha.com",
#     "www.instacart.com",
#     "www.kayak.com",
#     "fiscalnote.com",
#     "www.klarna.com",
#     "www.joinmilo.com",
#     "zapier.com",
# ]

# # Create the parent directory
# if not os.path.exists("ai-plugins"):
#     os.makedirs("ai-plugins")

# for site in tqdm(websites):
#     print(f'Working on {site}')
    
#     # Create a subfolder for each site
#     subfolder_path = f"ai-plugins/{site}"
#     if os.path.exists(subfolder_path):
#         shutil.rmtree(subfolder_path)
#     os.makedirs(subfolder_path)
    
#     # Fetch the ai-plugin.json
#     url = f"https://{site}/.well-known/ai-plugin.json"
#     response = requests.get(url)
#     if response.status_code == 200:
#         print(f'Found ai-plugin.json')
#         plugin_json = response.json()

#         # Save the ai-plugin.json to the respective subfolder
#         with open(f"{subfolder_path}/ai-plugin.json", "w") as file:
#             json.dump(plugin_json, file, indent=4)
        
#         # Create a new file for the endpoint pointed by api.url
#         api_url = plugin_json.get("api", {}).get("url")
#         if api_url:
#             # Taking the last part of the URL
#             last_segment = api_url.split('/')[-1]

#             # Sanitizing the last segment to create a valid filename
#             sanitized_filename = re.sub(r'[^a-zA-Z0-9\-]', '_', last_segment)
#             with open(f"{subfolder_path}/{sanitized_url}", "w") as file:
#                 file.write(f"This file represents the endpoint: {api_url}")


  0%|          | 0/7 [00:00<?, ?it/s]

Working on www.wolframalpha.com
Found ai-plugin.json
Working on www.instacart.com
Working on www.kayak.com
Working on fiscalnote.com
Working on www.klarna.com
Found ai-plugin.json
Working on www.joinmilo.com
Found ai-plugin.json
Working on zapier.com
Found ai-plugin.json


## Tidying up that code so that it is easier to improve

In [22]:
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm
import os
import json
import requests
import shutil
import re

def clear_existing_directory(path: str) -> None:
    """Clears the existing directory if it exists"""
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)

def fetch_plugin_json(site: str) -> dict:
    """Fetches the ai-plugin.json file from the site"""
    url = f"https://{site}/.well-known/ai-plugin.json"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return None

def save_json_to_file(json_obj: dict, filepath: str) -> None:
    """Saves a JSON object to a file"""
    with open(filepath, "w") as file:
        json.dump(json_obj, file, indent=4)

def fetch_api_data(api_url: str) -> str:
    """Fetches data from the API endpoint"""
    response = requests.get(api_url)
    if response.status_code == 200:
        return response.text
    return None

def create_api_endpoint_file(api_url: str, filepath: str) -> None:
    """Creates a new file with the data from the endpoint pointed by api.url"""
    api_data = fetch_api_data(api_url)
    if api_data:
        with open(filepath, "w") as file:
            file.write(api_data)

def fetch_and_save_plugin(site: str, parent_dir: str) -> None:
    """Fetches the ai-plugin.json from a site and saves it to a subfolder"""
    plugin_json = fetch_plugin_json(site)
    if plugin_json:
        subfolder_path = os.path.join(parent_dir, site)
        os.makedirs(subfolder_path)
        save_json_to_file(plugin_json, os.path.join(subfolder_path, "ai-plugin.json"))
        api_url = plugin_json.get("api", {}).get("url")
        if api_url:
            last_segment = api_url.split('/')[-1]
            sanitized_filename = re.sub(r'[^a-zA-Z0-9\-\.]', '_', last_segment)
            if sanitized_filename == "":
                sanitized_filename = "default_api_endpoint"
            create_api_endpoint_file(api_url, os.path.join(subfolder_path, sanitized_filename))


def update_plugins_info(websites):
    # Delete the ai-plugins directory if it already exists and create a new one
    clear_existing_directory("ai-plugins")

    # Use a ThreadPoolExecutor to fetch and save plugins from all websites simultaneously
    with ThreadPoolExecutor() as executor:
        list(tqdm(executor.map(fetch_and_save_plugin, websites, ["ai-plugins"]*len(websites)), total=len(websites)))
    

websites = [
    "www.wolframalpha.com",
    "www.instacart.com",
    "www.kayak.com",
    "fiscalnote.com",
    "www.klarna.com",
    "www.joinmilo.com",
    "zapier.com",
]

update_plugins_info(websites)


  0%|          | 0/7 [00:00<?, ?it/s]

## Further improvements:


Updates:
- I no longer want to delete the whole ai-plugins folder initially, from now on I only want to update what is already in there.
- If the content of the already saved files is exactly the same as what is obtained from the URLs, then no update needs to happen (the files do not need to be written again).
- If the content of the already saved files differs, then the existing files should be copied into a subfolder that includes the current date and time in the folder name (i.e. keep a record of the obsolete files) and then the up-to-date files should be overwritten with the new content.

In [13]:
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm
import os
import json
import requests
import shutil
import datetime
import re

def clear_existing_directory(path: str) -> None:
    """Clears the existing directory if it exists
    
    Note: Useful for testing to start from a clean slate. Not used otherwise
    """
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)

def fetch_plugin_json(site: str) -> dict:
    """Fetches the ai-plugin.json file from the site"""
    url = f"https://{site}/.well-known/ai-plugin.json"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return None

def save_json_to_file(json_obj: dict, filepath: str) -> None:
    """Saves a JSON object to a file"""
    with open(filepath, "w") as file:
        json.dump(json_obj, file, indent=4)

def fetch_api_data(api_url: str) -> str:
    """Fetches data from the API endpoint"""
    response = requests.get(api_url)
    if response.status_code == 200:
        return response.text
    return None

def create_api_endpoint_file(api_url: str, filepath: str) -> None:
    """Creates a new file with the data from the endpoint pointed by api.url"""
    api_data = fetch_api_data(api_url)
    if api_data:
        with open(filepath, "w") as file:
            file.write(api_data)

def read_json_from_file(filepath: str) -> dict:
    """Reads a JSON object from a file"""
    with open(filepath, "r") as file:
        return json.load(file)

def get_existing_file_content(filepath: str) -> str:
    """Returns the content of the file if it exists, otherwise returns None."""
    if os.path.exists(filepath):
        with open(filepath, "r") as file:
            return file.read()
    return None

def save_old_file(filepath: str, old_files_subfolder: str) -> None:
    """Saves the old file in a subfolder with the current date and time."""
    shutil.copy(filepath, old_files_subfolder)

def create_old_files_subfolder(parent_dir: str) -> str:
    """Creates a subfolder for old files with the current date and time."""
    old_files_subfolder = os.path.join(parent_dir, datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
    os.makedirs(old_files_subfolder, exist_ok=True)
    return old_files_subfolder

def fetch_and_save_plugin(site: str, parent_dir: str) -> None:
    """Fetches the ai-plugin.json from a site and saves it to a subfolder"""
    plugin_json = fetch_plugin_json(site)
    if plugin_json:
        subfolder_path = os.path.join(parent_dir, site)
        os.makedirs(subfolder_path, exist_ok=True)

        # Handle ai-plugin.json
        plugin_json_filepath = os.path.join(subfolder_path, "ai-plugin.json")
        existing_plugin_json = get_existing_file_content(plugin_json_filepath)
        new_plugin_json = json.dumps(plugin_json, indent=4)

        # Handle API endpoint
        api_url = plugin_json.get("api", {}).get("url")
        last_segment = api_url.split('/')[-1]
        sanitized_filename = re.sub(r'[^a-zA-Z0-9\-\.]', '_', last_segment)
        if sanitized_filename == "":
            sanitized_filename = "default_api_endpoint"
        api_filepath = os.path.join(subfolder_path, sanitized_filename)
        existing_api_data = get_existing_file_content(api_filepath)
        new_api_data = fetch_api_data(api_url)

        # Check if either file changed
        plugin_json_updated = False
        api_info_updated = False
        if not existing_plugin_json or json.loads(existing_plugin_json) != json.loads(new_plugin_json):
            print(f'New ai-plugin.json detected for {site}')
            plugin_json_updated = True
        if existing_api_data != new_api_data:
            print(f'New api data detected for {site}')
            api_info_updated = True
            
        # If either file has changed, create a subfolder for old files and save both old files to it
        if plugin_json_updated or api_info_updated:
            old_files_subfolder = create_old_files_subfolder(subfolder_path)
            if os.path.exists(plugin_json_filepath):
                save_old_file(plugin_json_filepath, old_files_subfolder)
            if os.path.exists(api_filepath):
                save_old_file(api_filepath, old_files_subfolder)

        # Save new data
        if plugin_json_updated:
            save_json_to_file(plugin_json, plugin_json_filepath)
        if api_info_updated:
            create_api_endpoint_file(api_url, api_filepath)

def update_plugins_info(websites):
    # Create the ai-plugins directory if it does not exist
    if not os.path.exists("ai-plugins"):
        os.makedirs("ai-plugins")

    # # Use a ThreadPoolExecutor to fetch and save plugins from all websites simultaneously
    # with ThreadPoolExecutor() as executor:
    #     list(tqdm(executor.map(fetch_and_save_plugin, websites, ["ai-plugins"]*len(websites)), total=len(websites)))
    
    for site in tqdm(websites):
        fetch_and_save_plugin(site, "ai-plugins")
    

websites = [
    "showme.redstarplugin.com",
    "www.wolframalpha.com",
    "www.instacart.com",
    "www.kayak.com",
    "fiscalnote.com",
    "www.klarna.com",
    "www.joinmilo.com",
    "zapier.com",
]

update_plugins_info(websites)


  0%|          | 0/8 [00:00<?, ?it/s]

New ai-plugin.json detected for showme.redstarplugin.com
New api data detected for www.wolframalpha.com


#  TODO:
- Why is wolfram apispec.json being detected as changed when it isn't
- I can json.loads, but what if it's a toml instead? 
- hmm...
- Add more plugin addresses! 

In [31]:
existing = get_existing_file_content('ai-plugins/www.wolframalpha.com/apispec.json')
new = fetch_api_data('https://www.wolframalpha.com/.well-known/apispec.json')
json.loads(existing) == json.loads(new)

True